In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import openpyxl
%matplotlib inline

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
pl_df = pd.read_csv('results.csv')

pl_df

In [ ]:
EPL_Data = pl_df.drop(range(0,(2824)), axis=0)
EPL_Data 

In [ ]:
pd.isna(EPL_Data).sum()

In [ ]:
EPL_Data['Month'] = pd.DatetimeIndex(EPL_Data['DateTime']).month
EPL_Data['Day'] = pd.DatetimeIndex(EPL_Data['DateTime']).day

In [ ]:
EPL_Data.drop(['DateTime'], axis=1)
EPL_Data.describe(include = 'O')

In [ ]:
matplotlib.rcParams['font.size'] = 18
matplotlib.rcParams['figure.figsize'] = (30, 10)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

Season-Wise Analysis

In [ ]:
EPL_by_Season = EPL_Data.groupby(['Season']).sum()

EPL_by_Season = EPL_by_Season.drop(['DateTime','HomeTeam', 'AwayTeam', 'FTR', 'HTR', 'Referee', 'Month', 'Day' ], axis=1)

EPL_by_Season

In [ ]:
EPL_by_Season['Goals'] = EPL_by_Season['FTHG'] + EPL_by_Season['FTAG']
EPL_by_Season = EPL_by_Season.reset_index()
sns.barplot(x='Goals', y='Season', data=EPL_by_Season)
plt.title('Number of Goals Scored Per Season') 
plt.ylabel('Goals')

In [ ]:
max_goals = EPL_by_Season['Goals'].max()
min_goals = EPL_by_Season['Goals'].min()

max_season = EPL_by_Season.loc[EPL_by_Season['Goals'] == max_goals, 'Season'].values[0]
min_season = EPL_by_Season.loc[EPL_by_Season['Goals'] == min_goals, 'Season'].values[0]

results = pd.DataFrame({'Season': [max_season, min_season], 
                        'Goals': [max_goals, min_goals], 
                        'Type': ['Max', 'Min']})

results

In [ ]:
EPL_melted = EPL_by_Season[['Season', 'FTHG', 'FTAG']]
EPL_melt = EPL_melted.melt("Season",var_name="H/A",value_name="Goals")

EPL_melt

In [ ]:
sns.barplot(x='Goals', y='Season', hue = 'H/A',data = EPL_melt)
plt.title('Number of Goals Scored Per Season') 
plt.ylabel('Season')
plt.legend(['Home Goals', 'Away Goals'])

In [ ]:
x = EPL_melt.groupby('H/A').sum(numeric_only=True).iloc[0]
y = EPL_melt.groupby('H/A').sum(numeric_only=True).iloc[1]

percentage_change = ((y - x) / abs(x) * 100)

In [ ]:
EPL_by_Season['Goals'] = EPL_by_Season['FTHG'] + EPL_by_Season['FTAG']
EPL_by_Season['Half_Goals'] = EPL_by_Season['HTHG'] + EPL_by_Season['HTAG']
EPL_by_Season['2nd_Half_Goals'] = EPL_by_Season['Goals'] - EPL_by_Season['Half_Goals']

In [ ]:
EPL_melted_half = EPL_by_Season[['Season', 'Half_Goals', '2nd_Half_Goals']]
EPL_melt_half = EPL_melted_half.melt("Season",var_name="1st_half/2nd_half",value_name="Goals")
EPL_melt_half

In [ ]:
sns.barplot(x='Goals', y='Season', hue = '1st_half/2nd_half',data = EPL_melt_half)
plt.title('Number of the 1st & 2nd Half Goals Scored Per Season') 
plt.ylabel('Season')
plt.legend(['1st Half Goals', '2nd Half Goals'])

In [ ]:
EPL_melt_half.groupby('1st_half/2nd_half').sum(numeric_only=True)

In [ ]:
x = EPL_melt_half.groupby('1st_half/2nd_half').sum(numeric_only=True).iloc[0]
y = EPL_melt_half.groupby('1st_half/2nd_half').sum(numeric_only=True).iloc[1]

percentage_change = (abs(y- x) / abs(x) * 100)
percentage_change

In [ ]:
EPL_heat = EPL_Data.groupby(['Season','Month']).sum(numeric_only=True).reset_index()
EPL_heat['Goals'] = EPL_heat['FTHG'] + EPL_heat['FTAG']
Final = EPL_heat.pivot(index='Season', columns='Month', values='Goals')
# month_order = ['8', '9', '10', '11', '12', '1', '2', '3', '4', '5', '6','7']
# Final = Final.reindex(columns=month_order)

In [ ]:
sns.heatmap(Final, annot=True, cmap='Greens', fmt='g')
plt.title('Goals Distribution based on month') 

In [ ]:
EPL_Data.groupby(['Season','Month']).sum(numeric_only=True)

In [ ]:
EPL_count = EPL_Data.groupby(['Season','Month']).count().reset_index()
EPL_heat['G/M'] = round(EPL_heat['Goals'] / EPL_count['FTHG'])
Final = EPL_heat.pivot(index='Season', columns='Month', values='G/M')
Final

In [ ]:
sns.heatmap(data=Final, annot = True, cmap = 'Blues',fmt='g')
plt.title('Goals per match based on month') 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
sns.lineplot(x = EPL_by_Season.Season, y = EPL_by_Season.HS, data = EPL_by_Season, legend='auto')
sns.lineplot(x = EPL_by_Season.Season, y = EPL_by_Season.AS, data = EPL_by_Season, legend='auto')
sns.lineplot(x = EPL_by_Season.Season, y = EPL_by_Season.HST, data = EPL_by_Season, legend='auto')
sns.lineplot(x = EPL_by_Season.Season, y = EPL_by_Season.AST, data = EPL_by_Season, legend='auto')
plt.ylabel('Shots')
plt.title('Home Shots vs Away Shots') 
# plt.xticks(rotation=60)

Team-Wise Analysis

In [ ]:
EPL_Data

In [ ]:
EPL_home = EPL_Data.groupby(EPL_Data['HomeTeam'])
EPL_away = EPL_Data.groupby(EPL_Data['AwayTeam'])

In [ ]:
EPL_team_goals = EPL_home['FTHG'].sum() + EPL_away['FTAG'].sum()
EPL_team_goals = EPL_team_goals.reset_index()
EPL_team_goals.rename(columns={0 :'Goals'}, inplace=True )
EPL_team_goals.rename(columns={'HomeTeam' :'Team'}, inplace=True )
EPL_team_goals = EPL_team_goals.sort_values(['Goals'], ascending=False)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (20, 15)
sns.barplot(x="Goals",y='Team', data = EPL_team_goals)
plt.title('Overall Goals Scored')
plt.ylabel('Team')

In [ ]:
EPL_team_HomeSCR = EPL_home[['FTHG', 'HS']].sum().reset_index() 
EPL_team_AwaySCR = EPL_away[['FTAG', 'AS']].sum().reset_index() 
EPL_team_HomeSCR.rename( columns={'HomeTeam' :'Team'}, inplace=True )
EPL_team_AwaySCR.rename( columns={'AwayTeam' :'Team'}, inplace=True )
EPL_team_SCR = pd.concat([EPL_team_HomeSCR, EPL_team_AwaySCR['FTAG'], EPL_team_AwaySCR['AS']], axis = 1)
EPL_team_SCR['SCR'] = (EPL_team_SCR['FTHG'] + EPL_team_SCR['FTAG']) / (EPL_team_SCR['AS'] + EPL_team_SCR['HS']) * 100
EPL_team_SCR

In [ ]:
EPL_team_SCR = EPL_team_SCR.sort_values(['SCR'], ascending=False)
matplotlib.rcParams['figure.figsize'] = (20, 20)
sns.barplot(x="SCR",y='Team', data = EPL_team_SCR )
plt.title('Shot Conversion Rate')
plt.ylabel('Team')

In [ ]:
home_win=100*round((EPL_Data.loc[EPL_Data['FTR']=='H'].groupby('HomeTeam')['FTR'].count()/EPL_Data.groupby('HomeTeam')['FTR'].count()),3)
home_win.sort_values(ascending = False)

In [ ]:
home_win.sort_values(ascending=True).plot(kind = 'barh')
plt.xlabel('Percentage')
plt.ylabel('Team')
plt.legend(['% Wins'])
plt.title("Home Ground Win %")

Referee-Wise Analysis

In [ ]:
EPL_ref = EPL_Data.groupby(['Referee']).sum(numeric_only=True).reset_index()
EPL_ref['RC'] = EPL_ref['HR'] + EPL_ref['AR']
EPL_ref = EPL_ref.loc[EPL_ref['RC'] != 0].sort_values(['RC'], ascending=False)
matplotlib.rcParams['figure.figsize'] = (20, 30)
sns.barplot(x="RC",y='Referee', data = EPL_ref )
plt.title('Red Cards given by Referees')
plt.ylabel('Referee')
plt.xlabel('Red Cards')

In [ ]:
# Get a list of all the unique referee names
referee_names = EPL_Data['Referee'].unique()

# Create an empty list to store the results
results = []

# Loop through each referee's name and perform the same analysis
for referee in referee_names:
    # Filter the data to select only the games where the referee was the current referee
    referee_games = EPL_Data[EPL_Data['Referee'] == referee]
    
    # Filter the data to select only the games where Arsenal played with the current referee
    arsenal_referee_games = referee_games[(referee_games['HomeTeam'] == 'Man United') | (referee_games['AwayTeam'] == 'Man United')]
    
    # Count the number of games played under the current referee
    num_games_referee = len(arsenal_referee_games)
    
    # Skip the current iteration if there were no games played under the current referee
    if num_games_referee == 0:
        continue
    
    # Count the number of games won under the current referee
    num_wins_referee = len(arsenal_referee_games[arsenal_referee_games['FTR'] == 'H'])
    
    # Calculate the percentage of games won under the current referee
    percent_wins_referee = num_wins_referee / num_games_referee * 100
    
    # Add the results to the list
    results.append({'Referee': referee, 'Num Games': num_games_referee, 'Num Wins': num_wins_referee, 'Percent Wins': round(percent_wins_referee, 2)})

# Convert the list of results to a pandas DataFrame object
results_df = pd.DataFrame(results)

# Sort the results DataFrame by the "Num Games" column in descending order
results_df = results_df.sort_values(by='Num Games', ascending=False)

# Print the result
print('Man United')
print(results_df)

In [ ]:
# Filter the data to select only the games where M Dean was the referee
dean_games = EPL_Data[EPL_Data['Referee'] == 'H Webb']

# Get a list of all the unique team names
team_names = dean_games['HomeTeam'].unique()

# Create an empty list to store the results
results = []

# Loop through each team's name and perform the same analysis
for team in team_names:
    # Filter the data to select only the games where the team was the current team
    team_games = dean_games[(dean_games['HomeTeam'] == team) | (dean_games['AwayTeam'] == team)]
    
    # Count the number of games played under the current referee
    num_games_team = len(team_games)
    
    # Skip the current iteration if there were no games played under the current referee
    if num_games_team == 0:
        continue
    
    # Count the number of games won, drawn, and lost under the current referee
    num_wins_team = len(team_games[team_games['FTR'] == 'H'])
    num_draws_team = len(team_games[team_games['FTR'] == 'D'])
    num_losses_team = len(team_games[team_games['FTR'] == 'A'])
    
    # Calculate the percentage of games won under the current referee
    percent_wins_team = num_wins_team / num_games_team * 100
    
    # Add the results to the list
    results.append({'Team': team, 'Num Games': num_games_team, 'Num Wins': num_wins_team, 'Num Draws': num_draws_team, 'Num Losses': num_losses_team, 'Percent Wins': round(percent_wins_team, 2)})

# Convert the list of results to a pandas DataFrame object
results_df = pd.DataFrame(results)

# Sort the results DataFrame by the "Num Games" column in descending order
results_df = results_df.sort_values(by='Num Games', ascending=False)

# Print the result
print('H Webb')
print(results_df.head(20))

Output Files

In [ ]:
with pd.ExcelFile('results.xlsx') as writer:
    EPL_heat.to_excel(writer, sheet_name='Goal Distro per month')